In [39]:
import os
import cv2
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
import sys
sys.path.append('path/to/yolov5')  # Add the YOLOv5 directory to Python path


Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs


Testing direct access to dataset...
Error accessing __getitem__: Sizes of tensors must match except in dimension 1. Expected size 76 but got size 75 for tensor number 1 in the list.

Testing DataLoader...


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 76 but got size 75 for tensor number 1 in the list.